**Description**: Shows how to compare MIP formulations for transportation problems with piecewise linear objective functions.

**Author**: Juan Pablo Vielma

**License**: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# MIP formulations for piecewise linear functions  with JuMP

We consider the transportation problem
$$\begin{alignat*}{3}
 &\min & \quad  \sum_{s=1}^{n_s} \sum_{d=1}^{n_d} f_{s,d}(x_{s,d})  \\
 \notag&s.t.\\
 &      &  \sum_{d=1}^{n_d} x_{s,d} &= supply_s,  &\quad& \forall s\in \{1,\ldots,n_s\} \\
  &      &   \sum_{s=1}^{n_s} x_{s,d} &= demand_d,  &\quad& \forall d\in \{1,\ldots,n_d\} \\
  &      &  x_{s,d} &\geq 0,  &\quad& \forall s\in \{1,\ldots,n_s\},\quad d\in \{1,\ldots,n_d\} 
 \end{alignat*}$$
 where $ f_{s,d}:\mathbb{R}\to \mathbb{R}$ is a piecewise linear function. 

We begin constructing a Linear Programming formulation for the case in which all $ f_{s,d}$ are linear and then consider Mixed Integer Programming formulations for the general case. 

In [1]:
using JuMP, Cbc

In [2]:
# Create JuMP model, using Cbc as the solver
model = Model(solver=CbcSolver())

# Data
snodes     = 3
supply     = rand(1:10,snodes)
dnodes     = 3
demand     = rand(1:10,dnodes)
demandimbalance = sum(demand) - sum(supply)
if demandimbalance > 0
    supply[snodes] += demandimbalance
else
    demand[dnodes] -= demandimbalance
end    
objcoeff  = rand(snodes,dnodes)

# Define variables
@variable(model, x[1:snodes,1:dnodes] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[s=1:snodes], sum(x[s,d] for d in 1:dnodes) == supply[s]
    demand_constraints[d=1:dnodes], sum(x[s,d] for s in 1:snodes) == demand[d]    
end)

# Define objective
@objective(model, Min, sum( objcoeff[s,d]*x[s,d] for  s in 1:snodes, d in 1:dnodes))

# Display Model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(x))

Min 0.010111918645785645 x[1,1] + 0.9309926800348636 x[1,2] + 0.09349288465262418 x[1,3] + 0.05987823186736296 x[2,1] + 0.6856079210299666 x[2,2] + 0.6652963667712235 x[2,3] + 0.7633662593295183 x[3,1] + 0.18994114538294649 x[3,2] + 0.01807956747590067 x[3,3]
Subject to
 x[1,1] + x[1,2] + x[1,3] = 4
 x[2,1] + x[2,2] + x[2,3] = 2
 x[3,1] + x[3,2] + x[3,3] = 4
 x[1,1] + x[2,1] + x[3,1] = 3
 x[1,2] + x[2,2] + x[3,2] = 3
 x[1,3] + x[2,3] + x[3,3] = 4
 x[i,j] ≥ 0 ∀ i ∈ {1,2,3}, j ∈ {1,2,3}

Solving...
Solver status: Optimal
Cost: 0.9982500399631242
Solution: 
[1.0 0.0 3.0; 2.0 0.0 0.0; 0.0 3.0 1.0]


In [3]:
# Generate supply, demand and piecewise linear objective
function generateData(snodes,dnodes,segments)
   
    supply     = rand(1:10,snodes)
    demand     = rand(1:10,dnodes)
    demandimbalance = sum(demand) - sum(supply)
    if demandimbalance > 0
        supply[snodes] += demandimbalance
    else
        demand[dnodes] -= demandimbalance
    end
    
    fvalues = [ Array{Float64}(segments+1) for i=1:snodes,j=1:dnodes ]
    dvalues = [ Array{Float64}(0) for i=1:snodes,j=1:dnodes ]
    for s in 1:snodes
        for d in 1:dnodes
            drange = min(supply[s],demand[d])
            delta = drange / (segments)
            dvalues[s,d] = [ (i-1)*delta for i=1:(segments+1) ]
            
            slopes = sort(rand(segments),rev=true)
            fvalues[s,d][1] = 0
            for i in 2:(segments+1)
                fvalues[s,d][i] = fvalues[s,d][i-1] + slopes[i-1]*(dvalues[s,d][i]-dvalues[s,d][i-1])
            end
        end
    end
    
    supply, demand, fvalues, dvalues
end

supply, demand, fvalues, dvalues = generateData(1,1,8)
snodes = length(supply); dnodes = length(demand);

In [4]:
# Plot a piecewise linear objective
using Plots
plot(dvalues[1,1],fvalues[1,1])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 
 
 
 
 y1

In [5]:
using PiecewiseLinearOpt

# Generate Data
supply, demand, fvalues, dvalues = generateData(1,2,3)
snodes = length(supply); dnodes = length(demand);

# Create JuMP model, using Cbc as the solver
model = Model(solver=CbcSolver())

# Define variables
@variable(model, x[1:snodes,1:dnodes] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[s=1:snodes], sum(x[s,d] for d in 1:dnodes) == supply[s]
    demand_constraints[d=1:dnodes], sum(x[s,d] for s in 1:snodes) == demand[d]    
end)

# Define objective
@objective(model, Min, sum(piecewiselinear(model, x[s,d], dvalues[s,d], fvalues[s,d]) for s in 1:snodes, d in 1:dnodes))

# Display Model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(x))


Min z_1 + z_2
Subject to
 x[1,1] + x[1,2] = 14
 x[1,1] = 4
 x[1,2] = 10
 λ_1[1] + λ_1[2] + λ_1[3] + λ_1[4] = 1
 1.3333333333333333 λ_1[2] + 2.6666666666666665 λ_1[3] + 4 λ_1[4] - x[1,1] = 0
 1.1765991085007015 λ_1[2] + 1.8627044970569226 λ_1[3] + 2.174035428895752 λ_1[4] - z_1 = 0
 λ_1[3] + λ_1[4] - y_1[1] ≤ 0
 λ_1[2] + λ_1[3] + λ_1[4] - y_1[1] ≥ 0
 λ_1[4] - y_1[2] ≤ 0
 λ_1[3] + λ_1[4] - y_1[2] ≥ 0
 λ_2[1] + λ_2[2] + λ_2[3] + λ_2[4] = 1
 3.3333333333333335 λ_2[2] + 6.666666666666667 λ_2[3] + 10 λ_2[4] - x[1,2] = 0
 2.037228147062386 λ_2[2] + 3.9175983027888224 λ_2[3] + 3.960017492446624 λ_2[4] - z_2 = 0
 λ_2[3] + λ_2[4] - y_2[1] ≤ 0
 λ_2[2] + λ_2[3] + λ_2[4] - y_2[1] ≥ 0
 λ_2[4] - y_2[2] ≤ 0
 λ_2[3] + λ_2[4] - y_2[2] ≥ 0
 x[i,j] ≥ 0 ∀ i ∈ {1}, j ∈ {1,2}
 0 ≤ λ_1[i] ≤ 1 ∀ i ∈ {1,2,3,4}
 y_1[i] ∈ {0,1} ∀ i ∈ {1,2}
 0 ≤ λ_2[i] ≤ 1 ∀ i ∈ {1,2,3,4}
 y_2[i] ∈ {0,1} ∀ i ∈ {1,2}
 0 ≤ z_1 ≤ 2.174035428895752
 0 ≤ z_2 ≤ 3.960017492446624

Solving...
Solver status: Optimal
Cost: 6.134052921342376

In [6]:
function solveModel(supply, demand, fvalues, dvalues, pwlformulation,MIPsolver=CbcSolver())
    # Create JuMP model, using Cbc as the solver
    model = Model(solver=MIPsolver)

    # Define variables
    @variable(model, x[1:snodes,1:dnodes] >= 0)

    # Add constraints
    @constraints(model, begin    
        supply_constraints[s=1:snodes], sum(x[s,d] for d in 1:dnodes) == supply[s]
        demand_constraints[d=1:dnodes], sum(x[s,d] for s in 1:snodes) == demand[d]    
    end)

    # Define objective
    @objective(model, Min, 
                           sum(piecewiselinear(model, x[s,d], dvalues[s,d], fvalues[s,d];method=pwlformulation) 
                                                                             for s in 1:snodes, d in 1:dnodes))
    
    println("Solving for ",pwlformulation," ...")
    tic()
    status = solve(model)
    solvetime=toq();
    println("Solver status: ", status)
    println("Cost: ", getobjectivevalue(model))
    println("Solve time: ",solvetime)

end

solveModel (generic function with 2 methods)

In [7]:
using Gurobi


# Generate Data
supply, demand, fvalues, dvalues = generateData(10,10,32)
snodes = length(supply); dnodes = length(demand);

# Solve for four formulations using Gurobi
for formulation in [:CC,:MC,:SOS2,:Incremental,:Logarithmic,:DisaggLogarithmic,:ZigZag,:ZigZagInteger]
    solveModel(supply, demand, fvalues, dvalues, formulation,GurobiSolver(TimeLimit=10,OutputFlag=0))
end

Solving for CC ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.99823941461748
Solve time: 2.195021192
Solving for MC ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.998239414617473
Solve time: 6.190181782
Solving for SOS2 ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.99823941461748
Solve time: 0.189062884
Solving for Incremental ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.998239414617494
Solve time: 3.490531457
Solving for Logarithmic ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.99823941461748
Solve time: 0.433009594
Solving for DisaggLogarithmic ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.99823941461748
Solve time: 0.68674553
Solving for ZigZag ...
Academic license - for non-commercial use only
Solver status: Optimal
Cost: 25.99823941461748
Solve time: 0.885945284
S